In [43]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
import pandas as pd
from src.gpt_prompt import get_sentiment_general, get_sentiment_general_parallel, get_risk_score
import datetime

In [44]:
df_news = pd.read_csv("../data/apple_2023/news_apple_2023.csv", index_col=0)
# df_news["date"] = pd.to_datetime(df_news['date'])

In [45]:
n = df_news.shape[0]
ind = 5
batch_size = 40

In [46]:
scores = []
batch = []
index = []
for i in range(n):
    if i % 10 == ind:
        index.append(i)
        batch.append(df_news["news"][i])
    # index.append(i)
    # batch.append(df_news["news"][i])
    if len(batch) >= batch_size:
        result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
        print(f"finish {i}")
        scores += result1
        batch = []  
        # try:
        #     result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
        #     print(f"finish {i}")
        #     scores += result1
        #     batch = []  
        # except:
        #     print(f"Fail {index[-1]}")
        #     index = index[:-batch_size]
        #     batch = [] 
            
            
if batch:
    result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
    print(f"finish {i}")
    scores += result1
    batch = []  
    # try:
    #     result1 = get_sentiment_general_parallel(batch,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False, threads=20)
    #     print(f"finish {i}")
    #     scores += result1
    #     batch = []  
    # except:
    #     print(f"Fail {index[-1]}")

        
    
    
    

 average running time: 0.20 second
finish 394
 average running time: 0.26 second
finish 794
 average running time: 0.18 second
finish 1194
 average running time: 0.18 second
finish 1594
 average running time: 0.18 second
finish 1994
 average running time: 0.44 second
finish 2394
 average running time: 0.19 second
finish 2794
 average running time: 0.22 second
finish 3194
 average running time: 0.19 second
finish 3594
 average running time: 0.25 second
finish 3994
 average running time: 0.95 second
finish 4394
 average running time: 0.18 second
finish 4794
 average running time: 0.19 second
finish 5194
 average running time: 0.22 second
finish 5594
 average running time: 0.33 second
finish 5994
 average running time: 0.19 second
finish 6394
 average running time: 1.13 second
finish 6794
 average running time: 0.17 second
finish 7194
 average running time: 0.18 second
finish 7594
 average running time: 0.42 second
finish 7994
 average running time: 0.20 second
finish 8394
 average runnin

ServiceUnavailableError: The server is overloaded or not ready yet.

In [47]:
def convert_response_to_score(response):
    dic = {"BULLISH": 1, "NEUTRAL": 0, "BEARISH":-1}
    scores = []
    for r in response:
        if r[0] not in dic:
            scores.append(5)
            continue
        scores.append(dic[r[0]])
        
    return scores

scores = convert_response_to_score(scores)
        

In [48]:
index = index[:len(scores)]
new_df = df_news.iloc[index].copy()
new_df["score"] = scores
new_df = new_df.reset_index(drop=True)
new_df.to_csv(f"../data/apple_2023/news_score_apple_2023_{ind+1}.csv")

In [49]:
new_df

,date,news,score
0,2023-01-01 03:10:16,Title: 5 appliances and smart home gadgets tha...,1
1,2023-01-01 08:07:23,Title: TechRadar's year in review: 2022 in pho...,0
2,2023-01-02 01:07:20,"Title: iPhone in 2023 — iPhone 15, iPhone 15 U...",1
3,2023-01-02 06:17:51,Title: 39%of this Apple Inc. (NASDAQ:AAPL) ins...,-1
4,2023-01-02 11:34:30,Title: Better Semiconductor Stock: TSMC vs. In...,0
...,...,...,...
1475,2023-05-19 08:11:33,Title: Apple Bans Some Staff From Using ChatGP...,-1
1476,2023-05-19 09:38:21,Title: Google Celebrates As Samsung Retracts P...,1
1477,2023-05-19 10:51:25,Title: Check Out What Whales Are Doing With AA...,1
1478,2023-05-19 11:41:26,Title: Everrati announces a new all-electric P...,1


In [50]:
dictionary = new_df.set_index('date')['score'].to_dict()
dictionary

{'2023-01-01 03:10:16': 1,
 '2023-01-01 08:07:23': 0,
 '2023-01-02 01:07:20': 1,
 '2023-01-02 06:17:51': -1,
 '2023-01-02 11:34:30': 0,
 '2023-01-02 19:23:24': -1,
 '2023-01-03 01:16:14': 1,
 '2023-01-03 05:56:26': 1,
 '2023-01-03 07:31:22': 0,
 '2023-01-03 08:38:30': 1,
 '2023-01-03 09:12:48': 1,
 '2023-01-03 10:01:29': 0,
 '2023-01-03 11:02:39': 1,
 '2023-01-03 12:12:29': 0,
 '2023-01-03 12:52:35': -1,
 '2023-01-03 13:41:27': 1,
 '2023-01-03 14:53:31': 1,
 '2023-01-03 16:03:21': 0,
 '2023-01-03 17:18:18': 1,
 '2023-01-03 18:05:26': 1,
 '2023-01-03 20:08:11': 1,
 '2023-01-03 21:01:17': -1,
 '2023-01-03 21:05:37': 1,
 '2023-01-03 23:43:05': 1,
 '2023-01-04 03:03:39': 1,
 '2023-01-04 05:18:13': 0,
 '2023-01-04 05:59:24': 0,
 '2023-01-04 06:52:59': 1,
 '2023-01-04 07:33:30': 0,
 '2023-01-04 08:32:41': -1,
 '2023-01-04 09:00:50': 0,
 '2023-01-04 09:37:59': 1,
 '2023-01-04 10:19:20': 1,
 '2023-01-04 11:16:35': 1,
 '2023-01-04 12:16:53': 1,
 '2023-01-04 13:33:27': 1,
 '2023-01-04 14:36:38':

In [51]:
def find_whole_minute_time_after_5_minutes(time):
    new_time = given_time + timedelta(minutes=5)

    # Adjust to the nearest whole minute
    if new_time.second > 0:
        new_time += timedelta(minutes=1)
        new_time = new_time.replace(second=0, microsecond=0)

In [52]:
# news = df["news"][0]
# print(get_sentiment_general(news,company="Apple",model="gpt-3.5-turbo-16k", output="polar", explanation=False)[0])
# print(get_sentiment_general(news,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False)[0])

In [53]:
# news = df["news"][::400]
# result1 = get_sentiment_general_parallel(news,company="Apple",model="gpt-3.5-turbo-16k", output="polar", explanation=False)
# print(result1)
# result2 = get_sentiment_general_parallel(news,company="Apple",model="gpt-3.5-turbo-1106", output="polar", explanation=False)
# print(result2)


In [54]:
# n = len(result1)
# c = 0
# for i in range(n):
#     if result1[i] == result2[i]:
#         c += 1
# print(c / n)
        